In [2]:
!pip install transformers datasets torch scikit-learn pandas

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('test-data/preprocessed.csv')
df = df.dropna(subset=['cleaned_text', 'target'])
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['target'], random_state=42)

In [8]:
from datasets import Dataset
train_ds = Dataset.from_pandas(train_df[['cleaned_text', 'target']].rename(columns={'cleaned_text': 'text', 'target': 'label'}))
test_ds = Dataset.from_pandas(test_df[['cleaned_text', 'target']].rename(columns={'cleaned_text': 'text', 'target': 'label'}))

In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

train_ds = train_ds.map(tokenize, batched=True)
test_ds = test_ds.map(tokenize, batched=True)
train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/168 [00:00<?, ? examples/s]

Map:   0%|          | 0/43 [00:00<?, ? examples/s]

In [14]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    logging_steps=20,
    save_strategy='no',
    report_to='none'
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
)
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
20,0.566200


TrainOutput(global_step=33, training_loss=0.419805519508593, metrics={'train_runtime': 11.6248, 'train_samples_per_second': 43.356, 'train_steps_per_second': 2.839, 'total_flos': 33151992975360.0, 'train_loss': 0.419805519508593, 'epoch': 3.0})

In [21]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

preds = trainer.predict(test_ds)
y_true = preds.label_ids
y_pred = np.argmax(preds.predictions, axis=1)

print(classification_report(y_true, y_pred, digits=3))
print(confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

           0      0.929     0.929     0.929        28
           1      0.867     0.867     0.867        15

    accuracy                          0.907        43
   macro avg      0.898     0.898     0.898        43
weighted avg      0.907     0.907     0.907        43

[[26  2]
 [ 2 13]]


In [22]:
model_save_path = './bert-finetuned'
trainer.save_model(model_save_path)